# 1. Loading the excel files

In [ ]:
import pandas as pd

df_1 = pd.read_excel('Company_Performance_File1.xlsx')
df_2 = pd.read_excel('Company_Performance_File2.xlsx')

In [ ]:
# 2. Cleaning Dataframe 1 (df_1)
# Converting ID to string and removing 'EMP' prefix and empty spaces
df_1['Employee_ID'] = df_1['Employee_ID'].astype(str).str.replace('EMP', '', case=False).str.strip()

# Removing duplicate Employee IDs
df_1 = df_1.drop_duplicates(subset=['Employee_ID'])

# Removing rows where Employee ID is missing
df_1_clean = df_1.dropna(subset=['Employee_ID']).copy()

# Filling missing Department values with 'Unknown'
df_1_clean['Department'] = df_1_clean['Department'].fillna('Unknown')

In [ ]:
# 3. Cleaning Dataframe 2 (df_2)
# Converting ID to string and removing 'EMP' prefix and empty spaces
df_2['Employee_ID'] = df_2['Employee_ID'].astype(str).str.replace('EMP', '', case=False).str.strip()

# Removing duplicate Employee IDs
df_2 = df_2.drop_duplicates(subset=['Employee_ID'])

# Removing rows where Employee ID is missing
df_2_clean = df_2.dropna(subset=['Employee_ID']).copy()

In [ ]:
# Converting Salary to numeric and filling missing values with mean
df_2_clean['Salary'] = pd.to_numeric(df_2_clean['Salary'], errors='coerce')
salary_mean = df_2_clean['Salary'].mean()
df_2_clean['Salary'] = df_2_clean['Salary'].fillna(salary_mean)

In [ ]:
# Cleaning Performance Score: Replacing 'Excellent' with 5 and handling NaNs
df_2_clean['Performance_Score'] = df_2_clean['Performance_Score'].replace('Excellent', 5)
df_2_clean['Performance_Score'] = pd.to_numeric(df_2_clean['Performance_Score'], errors='coerce').fillna(0)

In [ ]:
# 4. Merging the two dataframes (Outer Join to keep all records)
df_final = pd.merge(df_1_clean, df_2_clean, on='Employee_ID', how='outer')


In [ ]:
# 5. Post-Merge Cleaning and Sorting
# Converting Employee_ID back to numeric for proper sorting
df_final['Employee_ID'] = pd.to_numeric(df_final['Employee_ID'], errors='coerce')
df_final = df_final.dropna(subset=['Employee_ID'])

# Sorting data by Employee_ID in ascending order
df_final = df_final.sort_values(by='Employee_ID')

In [ ]:
# 6. Final Output
print(df_final.head())

# Saving the final cleaned report as V5
df_final.to_excel('Final_Performance_Report.xlsx', index=False)
print("Project Successfully Merged and Saved!")

In [ ]:
# --- START OF VISUALIZATION & FINAL CLEANING ---

# 1. Cleaning Names (Removing digits like Aisha1 -> Aisha)
df_final['Name'] = df_final['Name'].str.replace(r'\d+', '', regex=True).str.strip()

# 2. Filtering for Analysis (Excluding 'Unknown' departments)
analysis_df = df_final[df_final['Department'] != 'Unknown'].copy()

# 3. Grouping by Department for Average Salary
dept_salary = analysis_df.groupby('Department')['Salary'].mean().sort_values()

# 4. Drawing the Chart
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
dept_salary.plot(kind='barh', color='skyblue')
plt.title('Average Salary by Department')
plt.xlabel('Average Salary')
plt.ylabel('Department')
plt.tight_layout()

# Saving the chart
plt.savefig('Company_Performance_Chart.png')

print("\n[SUCCESS] Graph saved as Company_Performance_Chart.png")

# 5. Identifying Top 5 Performers
top_5 = df_final.sort_values(by='Performance_Score', ascending=False).head(5)
print("\n--- Top 5 Performers ---")
print(top_5[['Name', 'Department', 'Performance_Score']])

# Saving the final cleaned data to V5


with pd.ExcelWriter('Final_Company_Report.xlsx') as writer:
    df_final.to_excel(writer, sheet_name='Full_Report', index=False)
    top_5.to_excel(writer, sheet_name='Top_5_Performers', index=False)




print("[SUCCESS] Final Report  saved!")
